# **01-Paramétrage**

***libraries and variables***



In [5]:
# @title
from google.colab import drive
drive.mount('/content/OrangeOpen')
#------------------------------------------->
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")
DTNOW = pd.Timestamp.now().to_period('D')
DTNOW.to_timestamp().strftime("%Y%m%d")
#------------------------------------------->
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery
Id_Project='orangehlbdb'
bq = bigquery.Client(project=Id_Project)
Id_dataset_HLB=Id_Project+'.Orange_HLB'
Id_dataset_HLD=Id_Project+'.Orange_HLD'
BDD_HLB = bq.dataset(dataset_id=Id_dataset_HLB)
BDD_HLD = bq.dataset(dataset_id=Id_dataset_HLD)
job_config = bigquery.LoadJobConfig(autodetect= True)
#------------------------------------------->
Id_table_Production_HLB=Id_dataset_HLB+'.Tab_Production'
Id_table_Flux_HLB=Id_dataset_HLB+'.Tab_Flux'
Id_table_Flux_Intervalle_HLB=Id_dataset_HLB+'.Tab_Flux_Intervalle'
Id_table_RDV_HAGRID_HLB=Id_dataset_HLB+'.Tab_RDV_HAGRID'

Id_table_Production_HLD=Id_dataset_HLD+'.Tab_Production'

Drive already mounted at /content/OrangeOpen; to attempt to forcibly remount, call drive.mount("/content/OrangeOpen", force_remount=True).


***Recuperation des bases de données***

In [6]:
# @title
df_Effectif_Global_HLB=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/Effectif_Par_Jour_HLB.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_Effectif_Global_HLD=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/Effectif_Par_Jour_HLD.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_Date_Table=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/Date_Table.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_MT=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/MT.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_Jours_Ferie=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/Jours_Ferie.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')

df_Base_Data_agent_perf_Evita=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Data_agent_perf_Evita.csv' , delimiter=";" , decimal=",")
df_Base_Data_agent_Retrait_Evita=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Data_agent_Retrait_Evita.csv' , delimiter=";" , decimal=",")
df_Base_Appels_Courts=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Appels_Courts.csv' , delimiter=";" , decimal=",")
df_Base_PRA=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_PRA.csv' , delimiter=";" , decimal=",")
df_Base_REAL=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_REAL.csv' , delimiter=";" , decimal=",")
df_Base_HDOA=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_HLB_HDOA.csv' , delimiter=";" , decimal=",")
df_Base_CCA=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_HLB_CCA.csv' , delimiter=";" , decimal=",")
df_Base_MRC=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_MRC.csv' , delimiter=";" , decimal=",")
df_Base_Cubes_Appels=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Cubes_Appels.csv' , delimiter=";" , decimal=",")
df_Base_Data_Skills_Evita=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Data_Skills_Evita.csv' , delimiter=";" , decimal=",")
df_Base_Flux_Quotidien=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Flux_Quotidien.csv' , delimiter=";" , decimal=",")
df_Base_Rendez_Vous_Hagrid=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Rendez_Vous_Hagrid.csv' , delimiter=";" , decimal=",")
df_Base_Reit_HLD=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_HLD_Reiteration.csv' , delimiter=";" , decimal=",")
df_Base_Tracage_HLB=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_HLB_Tracage.csv' , delimiter=";" , decimal=",")

df_Base_PrevCapa_HLB=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/PrevCapa/PrevCapa_Jour_HLB.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_Base_PrevCapa_HLB_Intervalle=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/PrevCapa/PrevCapa_Intervalle_HLB.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')

# **02-Hot Line Boutique**

---


***HLB : Production Global***

In [7]:
# @title
df_Effectif_HLB = df_Effectif_Global_HLB[['Date_Format_1','CUID','Matricule']]
df_Production_HLB_1 = df_Base_Data_agent_Retrait_Evita.drop(columns=['Datation'])
df_Production_HLB_1 = pd.merge(left=df_Production_HLB_1, right=df_Effectif_HLB, how='left' , on=['Date_Format_1','CUID'])
df_Production_HLB_1.dropna(subset=['Matricule'], inplace=True)
df_Production_HLB_1 = df_Production_HLB_1[['Date_Format_1','CUID','Total_Logge','Duree_retrait_autres','Duree_retrait_formation','Duree_retrait_management','Duree_retrait_pause']]

df_Production_HLB_2 = df_Base_Data_agent_perf_Evita.loc[ (df_Base_Data_agent_perf_Evita['Flux'] == 'HLB') | (df_Base_Data_agent_perf_Evita['Flux'] == 'Help Desk') | (df_Base_Data_agent_perf_Evita['Flux'] == 'Autonomie') ]
df_Production_HLB_2.loc[df_Production_HLB_2['Flux'] == 'Help Desk' , 'Appels_HelpDesk'] = df_Production_HLB_2['Appels_traites']
df_Production_HLB_2.loc[df_Production_HLB_2['Flux'] == 'HLB' , 'Appels_HLB'] = df_Production_HLB_2['Appels_traites']
df_Production_HLB_2.loc[df_Production_HLB_2['Flux'] == 'Autonomie' , 'Appels_Autonomie'] = df_Production_HLB_2['Appels_traites']
df_Production_HLB_2 = df_Production_HLB_2[['Date_Format_1','CUID','Appels_HelpDesk','Appels_HLB','Appels_Autonomie','Appels_traites' , 'Appels_courts' , 'Appels_abandon_sur_sonnerie' , 'Appels_rotation_sur_sonnerie' , 'Appels_mis_en_attente' , 'Duree_log_total' , 'Duree_log_face_au_client' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_traitement' , 'Duree_conversation_appels_entrants' , 'Duree_garde_appels_entrants' , 'Duree_pas_libre' , 'Duree_wrapup' , 'Duree_pause_entre_appels' , 'Duree_THC']]

df_Effectif_HLB = df_Effectif_Global_HLB[['Date_Format_1','CUID','Log_PRA']]
df_Production_HLB_3 = df_Base_PRA.loc[ (df_Base_PRA['Flux'] == 'HLB') ]
df_Production_HLB_3 = df_Production_HLB_3.drop(columns=['Datation' ,'NOM','PRENOM','AFFECTATION'])
df_Production_HLB_3 = pd.merge(left=df_Production_HLB_3, right=df_Effectif_HLB, how='left' , left_on=['Date_Format_1','MATRICULE'], right_on=['Date_Format_1','Log_PRA'])
df_Production_HLB_3 = df_Production_HLB_3[['Date_Format_1','CUID','H_PLA','H_REA','H_ABS']]
df_Production_HLB_3.loc[df_Production_HLB_3['CUID'].isnull(), 'CUID'] = 'Non Defini'

pivoted=pd.pivot_table(df_Production_HLB_3, values='H_REA', index =['Date_Format_1'] , aggfunc='sum')
df_PRA_Dates=pd.DataFrame(pivoted.to_records())
df_PRA_Dates= df_PRA_Dates[df_PRA_Dates['H_REA'] == 0]
df_PRA_Dates=df_PRA_Dates["Date_Format_1"].unique()

df_Production_HLB_3_1 = df_Production_HLB_3.copy()
df_Production_HLB_3_1.rename(columns={'H_PLA': 'H_PLA_ABS','H_REA': 'H_REA_ABS','H_ABS': 'H_ABS_ABS'}, inplace=True)
df_Production_HLB_3_1 = df_Production_HLB_3_1[~df_Production_HLB_3_1['Date_Format_1'].isin(df_PRA_Dates)]

df_Production_HLB_4 = df_Base_REAL.loc[ ((df_Base_REAL['Flux'] == 'HLB') | (df_Base_REAL['Flux'] == 'Help Desk')) & (df_Base_REAL['Activite'] == 'Improd')]
df_Production_HLB_4 = pd.merge(left=df_Production_HLB_4, right=df_Effectif_HLB, how='left' , left_on=['Date_Format_1','MATRICULE'], right_on=['Date_Format_1','Log_PRA'])
df_Production_HLB_4 = df_Production_HLB_4[['Date_Format_1','CUID','DUREE']]

df_Production_HLB_5 = df_Base_HDOA[['Date_Format_1','CUID']]
df_Production_HLB_5['Tracage_HDOA']=1

df_Production_HLB_6 = df_Base_Appels_Courts.loc[ (df_Base_Appels_Courts['Flux'] == 'HLB') | (df_Base_Appels_Courts['Flux'] == 'Help Desk') | (df_Base_Appels_Courts['Flux'] == 'Autonomie') ]
df_Production_HLB_6.rename(columns={'Appels_Courts': 'Appels_Courts_Global' }, inplace=True)
df_Production_HLB_6.loc[df_Production_HLB_6['Flux'] == 'Help Desk' , 'Appels_Courts_HelpDesk'] = df_Production_HLB_6['Appels_Courts_Global']
df_Production_HLB_6.loc[df_Production_HLB_6['Flux'] == 'HLB' , 'Appels_Courts_HLB'] = df_Production_HLB_6['Appels_Courts_Global']
df_Production_HLB_6.loc[df_Production_HLB_6['Flux'] == 'Autonomie' , 'Appels_Courts_Autonomie'] = df_Production_HLB_6['Appels_Courts_Global']
df_Production_HLB_6 = df_Production_HLB_6[['Date_Format_1','CUID','Appels_Courts_Global','Appels_Courts_HelpDesk','Appels_Courts_HLB','Appels_Courts_Autonomie']]

df_Production_HLB_7 = df_Base_Rendez_Vous_Hagrid[['Date_Format_1','Conseiller intervenant','Nb Rdv total','NbRdvTotalClos','NbRdvTotalAnnule','Nb respect engagement']]
df_Production_HLB_7.rename(columns={'Conseiller intervenant': 'CUID' , 'Nb Rdv total': 'Nb_Rdv_total'  , 'Nb respect engagement': 'Nb_respect_engagement'  }, inplace=True)

df_Production_HLB_8 = df_Base_Rendez_Vous_Hagrid.loc[ (df_Base_Rendez_Vous_Hagrid['Motif clôture'] == 'Client injoignable') ]
df_Production_HLB_8 = df_Production_HLB_8[['Date_Format_1','Conseiller intervenant','Nb Rdv total','NbRdvTotalClos']]
df_Production_HLB_8.rename(columns={'Conseiller intervenant': 'CUID' , 'Nb Rdv total': 'Nb_Rdv_total_Inj'  , 'NbRdvTotalClos': 'NbRdvTotalClos_Inj'  }, inplace=True)

df_Production_HLB_9 = df_Base_CCA[['Date_Format_1','CUID']]
df_Production_HLB_9['Tracage_CCA']=1

df_Production_HLB_10 = df_Base_MRC[['Date_Format_1','CUID','NbreRdv','NbreRdvManques','NbreRdvSatisfaits']]
df_Production_HLB_10 = df_Production_HLB_10[(df_Production_HLB_10['Date_Format_1'] < int(DTNOW.to_timestamp().strftime("%Y%m%d")))]

df_Production_HLB_11 = df_Base_Tracage_HLB[['Date_Format_1','code_alliance']]
df_Production_HLB_11.rename(columns={'code_alliance': 'CUID'}, inplace=True)
df_Production_HLB_11['Tracage_HLB']=1

dfg_Production_HLB = pd.concat([df_Production_HLB_1, df_Production_HLB_2, df_Production_HLB_3, df_Production_HLB_3_1, df_Production_HLB_4, df_Production_HLB_5,df_Production_HLB_6,df_Production_HLB_7,df_Production_HLB_8,df_Production_HLB_9,df_Production_HLB_10,df_Production_HLB_11], join='outer')
t_table_Production_HLB = pd.pivot_table(dfg_Production_HLB, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
dfg_Production_HLB=pd.DataFrame(t_table_Production_HLB.to_records())

df_Effectif_HLB = df_Effectif_Global_HLB.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])
dfg_Production_HLB = pd.merge(left=dfg_Production_HLB, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Production_HLB = pd.merge(left=dfg_Production_HLB, right=df_Effectif_HLB, how='left' , on=['Date_Format_1','CUID'])

dfg_Production_HLB.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Production_HLB.csv' , index=False, sep = ';' , decimal=",")

dfg_Production_HLB['Datation']=dfg_Production_HLB['Datation'].values.astype(str)
dfg_Production_HLB['Datation'] = dfg_Production_HLB['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

table = bigquery.Table(Id_table_Production_HLB)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Production_HLB, table, job_config=job_config)

#Rendez_Vous_Hagrid-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

df_RDV_HLB_1 = df_Base_Rendez_Vous_Hagrid[['Date_Format_1','Motif clôture','Conseiller intervenant','Nb Rdv total','NbRdvTotalClos','NbRdvTotalAnnule','Nb respect engagement','DelaiTraitement']]
df_RDV_HLB_1.rename(columns={'Conseiller intervenant': 'CUID' , 'Nb Rdv total': 'Nb_Rdv_total'  , 'Nb respect engagement': 'Nb_respect_engagement' , 'Motif clôture': 'Motif_cloture'   }, inplace=True)

df_Effectif_HLB = df_Effectif_Global_HLB.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])
df_RDV_HLB_1 = pd.merge(left=df_RDV_HLB_1, right=df_Date_Table, how='left' , on='Date_Format_1')
df_RDV_HLB_1 = pd.merge(left=df_RDV_HLB_1, right=df_Effectif_HLB, how='left' , on=['Date_Format_1','CUID'])

df_RDV_HLB_1['Datation']=df_RDV_HLB_1['Datation'].values.astype(str)
df_RDV_HLB_1['Datation'] = df_RDV_HLB_1['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

table = bigquery.Table(Id_table_RDV_HAGRID_HLB)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(df_RDV_HLB_1, table, job_config=job_config)

#Annexe-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

DNNN = pd.Timestamp.now().to_period('D')
DNN = pd.Timestamp.now().to_period('D') - 40
DNN.to_timestamp().strftime("%Y%m%d")
dfg_Production_HLB['Activite_1']='ORANGE_HLB'
dfg_Production_HLB['Activite_2']='PAL_ORANGE HLB'
dfg_Production_HLB['Activite_3']='PAL_ORANGE HLB'
dfg_Production_HLB['Duree_Traitement']=dfg_Production_HLB['Duree_log_face_au_client_hors_attente'] /3600
dfg_Production_HLB['Connecte'] =  dfg_Production_HLB['Duree_log_total'] /3600
dfg_Production_HLB['Formation'] =  dfg_Production_HLB['Duree_retrait_formation'] /3600
dfg_Production_HLB['Manager'] =  dfg_Production_HLB['Duree_retrait_management'] /3600
dfg_Production_HLB['MYT'] =  dfg_Production_HLB['Duree_retrait_management'] /3600
dfg_Production_HLB['Dispo'] =  dfg_Production_HLB['Duree_dispo_CC'] /3600
dfg_Production_HLB['Pause'] =dfg_Production_HLB['Connecte'] -  dfg_Production_HLB['Duree_Traitement']  - dfg_Production_HLB['Dispo'] - dfg_Production_HLB['Manager'] - dfg_Production_HLB['Formation']
dfg_Production_HLB['CA'] =  dfg_Production_HLB['Appels_traites'] * 2.66

#DATA_AGENT-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

df_DATA_AGENT = dfg_Production_HLB[['Semaine_Format_2','Date_Format_1','Datation' , 'Log_PRA', 'Equipe', 'CDP', 'Localiastion', 'Activite_1', 'Appels_traites','Duree_Traitement','Connecte','Formation','Manager','Pause','Dispo','MYT','CA']]
df_DATA_AGENT.rename(columns = {'Semaine_Format_2':'Semaine','Datation':'Date', 'Log_PRA':'Matricule', 'Equipe':'Equipe', 'CDP':'CDP', 'Localiastion':'Localiastion', 'Activite_1':'Activite', 'Appels_traites':'Acte_Traite', 'Duree_Traitement':'Duree_Traitement', 'Connecte':'Heures_Connecte', 'Formation':'Heures_FC', 'Manager':'Heures_Brief', 'Pause':'Heures_Pause', 'Dispo':'Heures_Dispo', 'MYT':'MYT', 'CA':'CA'}, inplace = True)
df_DATA_AGENT['Affectation_Invison']='PAL_ORANGE HLB'
df_DATA_AGENT['Job_Code']='MAPALORGCC8'
df_DATA_AGENT['Objectif_Production']='4,5'
df_DATA_AGENT['Objectif_Qualite']=''
df_DATA_AGENT['Objectif_Commerce']=''
df_DATA_AGENT['Semaine'] = df_DATA_AGENT['Semaine'].str.replace('em','')
df_DATA_AGENT.sort_values(by=['Date'], inplace=True, ascending=True)
df_DATA_AGENT.dropna(subset=['Matricule'], inplace=True)
#df_DATA_AGENT = df_DATA_AGENT[(df_DATA_AGENT['Date_Format_1'] > int(DNN.to_timestamp().strftime("%Y%m%d"))) & (df_DATA_AGENT['Date_Format_1'] < int(DNNN.to_timestamp().strftime("%Y%m%d")))]
df_DATA_AGENT = df_DATA_AGENT.drop(columns=['Date_Format_1'])

Id_Project='palmier-data-orange'
Id_dataset=Id_Project+'.DATA_AGENT'
Id_table_DATA_AGENT_ORANGE_HLB=Id_dataset+'.DATA_AGENT_ORANGE_HLB'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_DATA_AGENT_ORANGE_HLB)
bq.delete_table(Id_table_DATA_AGENT_ORANGE_HLB, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(df_DATA_AGENT, Id_table_DATA_AGENT_ORANGE_HLB, job_config=job_config)

df_DATA_AGENT = df_DATA_AGENT.drop(columns=['Affectation_Invison','Job_Code','Objectif_Production','Objectif_Qualite','Objectif_Commerce','Semaine'])
df_DATA_AGENT.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_HLB_DATA_AGENT.csv' , index=False, sep = ';' , decimal=",")


#DATA_ScoreCard------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

DATA_ScoreCard_1 = dfg_Production_HLB[['Date_Format_1', 'Log_PRA', 'Activite_2', 'Activite_3','Connecte','Pause','Manager','Formation','Dispo','CA', 'Appels_traites','Nb_Rdv_total','Nb_respect_engagement','Appels_HelpDesk','Appels_Autonomie','Tracage_HDOA','Tracage_CCA']]
DATA_ScoreCard_1.rename(columns = {'Log_PRA':'Matricule', 'Activite_2':'Activite', 'Activite_3':'Campagne', 'Connecte':'Heures_Connecte', 'Pause':'Heures_Pause', 'Manager':'Heures_Brief', 'Formation':'Heures_FC', 'Dispo':'Heures_Dispo', 'CA':'Chiffre_Affaire', 'Appels_traites':'Appels_traitees', 'Nb_Rdv_total':'Nb_Rdv_total', 'Nb_respect_engagement':'Nb_pris_Eng', 'Appels_HelpDesk':'APP_Help_Desk', 'Appels_Autonomie':'APP_Autonomie', 'Tracage_HDOA':'Tracage_Help_Desk', 'Tracage_CCA':'Tracage_Autonomie'}, inplace = True)

DATA_ScoreCard_2 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'HLB') | (df_Base_Cubes_Appels['Flux'] == 'Help Desk') | (df_Base_Cubes_Appels['Flux'] == 'Autonomie') ]
DATA_ScoreCard_2 = DATA_ScoreCard_2[['Date_Format_1' , 'Presentes', 'Traites']]
DATA_ScoreCard_2.rename(columns = {'Presentes':'Appels_Recus_G', 'Traites':'Appels_Traite_G'}, inplace = True)
DATA_ScoreCard_2['Matricule'] =  'Global'
DATA_ScoreCard_2['Activite'] =  'PAL_ORANGE HLB'
DATA_ScoreCard_2['Campagne'] =  'PAL_ORANGE HLB'

DATA_ScoreCard_3 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'HLB') | (df_Base_Cubes_Appels['Flux'] == 'Help Desk') | (df_Base_Cubes_Appels['Flux'] == 'Autonomie')]
DATA_ScoreCard_3 = DATA_ScoreCard_3[['Date_Format_1' , 'Courts']]
DATA_ScoreCard_3.rename(columns = {'Courts':'Appels_Courts_G'}, inplace = True)
DATA_ScoreCard_3['Matricule'] =  'Global'
DATA_ScoreCard_3['Activite'] =  'PAL_ORANGE HLB'
DATA_ScoreCard_3['Campagne'] =  'PAL_ORANGE HLB'

df_DATA_ScoreCard = pd.concat([DATA_ScoreCard_1, DATA_ScoreCard_2, DATA_ScoreCard_3], join='outer')
t_table_DATA_ScoreCard = pd.pivot_table(df_DATA_ScoreCard , index=['Date_Format_1','Matricule','Activite','Campagne'],aggfunc = 'sum')
df_DATA_ScoreCard=pd.DataFrame(t_table_DATA_ScoreCard.to_records())

df_Date_Table_ScoreCard = df_Date_Table[['Date_Format_1', 'Datation']]
df_DATA_ScoreCard = pd.merge(left=df_DATA_ScoreCard, right=df_Date_Table_ScoreCard, how='left' , on='Date_Format_1')
df_DATA_ScoreCard.rename(columns = {'Datation':'Date'}, inplace = True)

df_DATA_ScoreCard = df_DATA_ScoreCard[(df_DATA_ScoreCard['Date_Format_1'] > int(DNN.to_timestamp().strftime("%Y%m%d"))) & (df_DATA_ScoreCard['Date_Format_1'] < int(DNNN.to_timestamp().strftime("%Y%m%d")))]
df_DATA_ScoreCard = df_DATA_ScoreCard.drop(columns=['Date_Format_1'])

df_DATA_ScoreCard = df_DATA_ScoreCard[['Date' , 'Matricule' , 'Activite' , 'Campagne' , 'Heures_Connecte' , 'Heures_Pause' , 'Heures_Brief' , 'Heures_FC' , 'Heures_Dispo' , 'Chiffre_Affaire' , 'Appels_traitees' , 'Nb_Rdv_total' , 'Nb_pris_Eng' , 'APP_Help_Desk' , 'APP_Autonomie' , 'Tracage_Help_Desk' , 'Tracage_Autonomie' , 'Appels_Recus_G' , 'Appels_Traite_G' , 'Appels_Courts_G']]

df_DATA_ScoreCard.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/orange_hlb_'+DNNN.to_timestamp().strftime("%d%m%Y")+'.csv' , index=False, sep = ';' , decimal=",")


***HLB : Flux***

In [8]:
# @title
#Flux_Globale-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

from datetime import datetime
df_Flux_1 = df_Base_PrevCapa_HLB[['Date_Format_1', 'Flux' , 'Commande','Capacitaire']]
df_Flux_1['Prevision'] = df_Flux_1['Commande'] * (1/0.9)

df_Flux_2 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'HLB') | (df_Base_Cubes_Appels['Flux'] == 'Help Desk') | (df_Base_Cubes_Appels['Flux'] == 'Autonomie') ]
df_Flux_2 = df_Flux_2[['Date_Format_1' ,'Flux', 'Presentes', 'Traites']]

df_JF = df_Jours_Ferie['Date_Format_1']
df_Flux_2 = df_Flux_2[~df_Flux_2['Date_Format_1'].isin(df_JF)]

df_Flux_3 = df_Base_Flux_Quotidien.loc[ (df_Base_Flux_Quotidien['Flux'] == 'HLB') | (df_Base_Flux_Quotidien['Flux'] == 'Help Desk') | (df_Base_Flux_Quotidien['Flux'] == 'Autonomie')]
df_Flux_3 = df_Flux_3[['Date_Format_1' ,'Flux' , 'Dissuades', 'Abandons', 'DMAbd', 'DMR' , 'Traites']]
df_Flux_3['DR'] = df_Flux_3['DMR'] * df_Flux_3['Traites']
df_Flux_3['DAbd'] = df_Flux_3['DMAbd'] * df_Flux_3['Abandons']
df_Flux_3 = df_Flux_3.drop(columns=['Traites' , 'DMAbd', 'DMR'])

df_Flux_4 = df_Base_Data_Skills_Evita.loc[ (df_Base_Data_Skills_Evita['Flux'] == 'HLB') | (df_Base_Data_Skills_Evita['Flux'] == 'Help Desk') | (df_Base_Data_Skills_Evita['Flux'] == 'Autonomie') ]
df_Flux_4 = df_Flux_4[['Date_Format_1' ,'Flux' , 'Appels_traites' , 'Appels_courts' , 'Appels_abandon_sur_sonnerie' , 'Appels_rotation_sur_sonnerie' , 'Appels_mis_en_attente' , 'Duree_log_total' , 'Duree_log_face_au_client' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_traitement' , 'Duree_conversation_appels_entrants' , 'Duree_garde_appels_entrants' , 'Duree_pas_libre' , 'Duree_wrapup' , 'Duree_pause_entre_appels' , 'Duree_THC']]

df_Flux_5 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'HLB') | (df_Base_Cubes_Appels['Flux'] == 'Help Desk') | (df_Base_Cubes_Appels['Flux'] == 'Autonomie')]
df_Flux_5 = df_Flux_5[['Date_Format_1' ,'Flux' , 'Courts']]

df_Flux_6 = df_Base_Rendez_Vous_Hagrid.loc[ (df_Base_Rendez_Vous_Hagrid['Motif clôture'] != 'Client injoignable') ]
df_Flux_6 = df_Flux_6[['Date_Format_1','Conseiller intervenant','Nb Rdv total']]
df_Flux_6.rename(columns={'Conseiller intervenant': 'CUID' , 'Nb Rdv total': 'Nb_Rdv_total' }, inplace=True)
df_Flux_6['Flux']='HLB'

dfg_Flux = pd.concat([df_Flux_1, df_Flux_2, df_Flux_3, df_Flux_4, df_Flux_5,df_Flux_6], join='outer')
t_table_Flux = pd.pivot_table(dfg_Flux , index=['Date_Format_1' , 'Flux'],aggfunc = 'sum')
dfg_Flux=pd.DataFrame(t_table_Flux.to_records())

dfg_Flux = pd.merge(left=dfg_Flux, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Flux.loc[dfg_Flux['Jour'] == 'Dimanche', 'Presentes'] = 0

DN = pd.Timestamp.now().to_period('D')
DN.to_timestamp().strftime("%Y%m%d")
dfg_Flux = dfg_Flux[(dfg_Flux['Date_Format_1'] < int(DN.to_timestamp().strftime("%Y%m%d")))]

dfg_Flux.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_HLB_Flux_Jours.csv' , index=False, sep = ';' , decimal=",")

dfg_Flux['Datation']=dfg_Flux['Datation'].values.astype(str)
dfg_Flux['Datation'] = dfg_Flux['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

table = bigquery.Table(Id_table_Flux_HLB)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Flux, table, job_config=job_config)

#DATA_PROD-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

DNN = pd.Timestamp.now().to_period('D') - 8
DNN.to_timestamp().strftime("%Y%m%d")

df_Data_Prod= dfg_Flux[['Date_Format_1','Datation' ,'Prevision', 'Presentes' , 'Traites', 'Duree_log_face_au_client_hors_attente' , 'Appels_traites','Capacitaire']]
df_Data_Prod['Flux']= 'HLB'
df_Data_Prod['Activite']= 'HLB'
df_Data_Prod['Job_Code']='MAPALORGCC8'
t_table_Data_Prod = pd.pivot_table(df_Data_Prod , index=['Date_Format_1','Datation' , 'Flux', 'Activite','Job_Code'],aggfunc = 'sum')
df_Data_Prod=pd.DataFrame(t_table_Data_Prod.to_records())

df_Data_Prod.loc[(df_Data_Prod['Appels_traites'] > 0) , 'DMT'] = df_Data_Prod['Duree_log_face_au_client_hors_attente'] / df_Data_Prod['Appels_traites']
df_Data_Prod.loc[(df_Data_Prod['Traites'] > 0) , 'QP'] = df_Data_Prod['Presentes'] / df_Data_Prod['Prevision']
df_Data_Prod.loc[(df_Data_Prod['Traites'] > 0) , 'QS'] = df_Data_Prod['Traites'] / df_Data_Prod['Presentes']
df_Data_Prod.loc[(df_Data_Prod['Traites'] > 0) , 'QE'] = df_Data_Prod['Traites'] / df_Data_Prod['Prevision']

df_Data_Prod = df_Data_Prod.drop(columns=['Duree_log_face_au_client_hors_attente', 'Appels_traites'])

df_Data_Prod= df_Data_Prod[['Job_Code','Datation' , 'Flux', 'Activite','Prevision','Capacitaire', 'Presentes' , 'Traites','DMT', 'QP', 'QS', 'QE']]
df_Data_Prod.rename(columns = {'Datation':'date', 'Prevision':'Forcast', 'Presentes':'Recu', 'Traites':'Traite'}, inplace = True)

#df_Data_Prod = df_Data_Prod[(df_Data_Prod['Date_Format_1'] > int(DNN.to_timestamp().strftime("%Y%m%d")))]
#df_Data_Prod = df_Data_Prod.drop(columns=['Date_Format_1'])

Id_Project='palmier-data-orange'
Id_dataset=Id_Project+'.DATA_PROD'
Id_table_DATA_PROD_ORANGE_HLB=Id_dataset+'.DATA_PROD_ORANGE_HLB'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_DATA_PROD_ORANGE_HLB)
bq.delete_table(Id_table_DATA_PROD_ORANGE_HLB, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(df_Data_Prod, Id_table_DATA_PROD_ORANGE_HLB, job_config=job_config)

df_Data_Prod = df_Data_Prod.drop(columns=['Job_Code'])
df_Data_Prod.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_HLB_DATA_PROD.csv' , index=False, sep = ';' , decimal=",")

#Flux_Intervalle-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

df_Flux_Intervalle_1 = df_Base_PrevCapa_HLB_Intervalle[['Date_Format_1', 'Demiheure', 'Flux', 'Commande']]
df_Flux_Intervalle_1['Prevision'] = df_Flux_Intervalle_1['Commande'] * (1/0.9)

df_Flux_Intervalle_2 = df_Base_Cubes_Appels.loc[  (df_Base_Cubes_Appels['Flux'] == 'HLB') | (df_Base_Cubes_Appels['Flux'] == 'Help Desk') | (df_Base_Cubes_Appels['Flux'] == 'Autonomie') ]
df_Flux_Intervalle_2 = df_Flux_Intervalle_2[['Date_Format_1' , 'Demiheure' ,'Flux' , 'Presentes', 'Traites']]

df_Flux_Intervalle_2 = df_Flux_Intervalle_2[~df_Flux_Intervalle_2['Date_Format_1'].isin(df_JF)]

df_Flux_Intervalle_3 = df_Base_Flux_Quotidien.loc[ (df_Base_Flux_Quotidien['Flux'] == 'HLB') | (df_Base_Flux_Quotidien['Flux'] == 'Help Desk') | (df_Base_Flux_Quotidien['Flux'] == 'Autonomie')]
df_Flux_Intervalle_3 = df_Flux_Intervalle_3[['Date_Format_1' , 'Demiheure' ,'Flux', 'Dissuades', 'Abandons', 'DMAbd', 'DMR' , 'Traites']]
df_Flux_Intervalle_3['DR'] = df_Flux_Intervalle_3['DMR'] * df_Flux_Intervalle_3['Traites']
df_Flux_Intervalle_3['DAbd'] = df_Flux_Intervalle_3['DMAbd'] * df_Flux_Intervalle_3['Abandons']
df_Flux_Intervalle_3 = df_Flux_Intervalle_3.drop(columns=['Traites' , 'DMAbd', 'DMR'])

df_Flux_Intervalle_4 = df_Base_Data_Skills_Evita.loc[ (df_Base_Data_Skills_Evita['Flux'] == 'HLB') | (df_Base_Data_Skills_Evita['Flux'] == 'Help Desk') | (df_Base_Data_Skills_Evita['Flux'] == 'Autonomie')  ]
df_Flux_Intervalle_4 = df_Flux_Intervalle_4[['Date_Format_1','Demi_heure' ,'Flux' , 'Appels_traites' , 'Appels_courts' , 'Appels_abandon_sur_sonnerie' , 'Appels_rotation_sur_sonnerie' , 'Appels_mis_en_attente' , 'Duree_log_total' , 'Duree_log_face_au_client' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_traitement' , 'Duree_conversation_appels_entrants' , 'Duree_garde_appels_entrants' , 'Duree_pas_libre' , 'Duree_wrapup' , 'Duree_pause_entre_appels' , 'Duree_THC']]
df_Flux_Intervalle_4.rename(columns = {'Demi_heure':'Demiheure'}, inplace = True)

df_Flux_Intervalle_5 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'HLB') | (df_Base_Cubes_Appels['Flux'] == 'Help Desk') | (df_Base_Cubes_Appels['Flux'] == 'Autonomie')  ]
df_Flux_Intervalle_5 = df_Flux_Intervalle_5[['Date_Format_1' ,'Demiheure', 'Flux' , 'Courts']]

dfg_Flux_Intervalle = pd.concat([df_Flux_Intervalle_1, df_Flux_Intervalle_2, df_Flux_Intervalle_3, df_Flux_Intervalle_4, df_Flux_Intervalle_5], join='outer')
t_table_Flux_Intervalle = pd.pivot_table(dfg_Flux_Intervalle , index=['Date_Format_1' ,'Demiheure', 'Flux'],aggfunc = 'sum')
dfg_Flux_Intervalle=pd.DataFrame(t_table_Flux_Intervalle.to_records())

dfg_Flux_Intervalle = pd.merge(left=dfg_Flux_Intervalle, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Flux_Intervalle = pd.merge(left=dfg_Flux_Intervalle, right=df_MT, how='left' , on='Demiheure')

dfg_Flux_Intervalle = dfg_Flux_Intervalle[(dfg_Flux_Intervalle['Date_Format_1'] < int(DN.to_timestamp().strftime("%Y%m%d")))]

dfg_Flux_Intervalle.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_HLB_Flux_Jours_Intervalle.csv' , index=False, sep = ';' , decimal=",")

dfg_Flux_Intervalle['Datation']=dfg_Flux_Intervalle['Datation'].values.astype(str)
dfg_Flux_Intervalle['Datation'] = dfg_Flux_Intervalle['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

table = bigquery.Table(Id_table_Flux_Intervalle_HLB)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Flux_Intervalle, table, job_config=job_config)

# **03-Hot Line Distributeur**

---


***HLD : Production Global***

In [9]:
# @title
df_Effectif_HLD = df_Effectif_Global_HLD[['Date_Format_1','CUID','Matricule']]
df_Production_HLD_1 = df_Base_Data_agent_Retrait_Evita.drop(columns=['Datation'])
df_Production_HLD_1 = pd.merge(left=df_Production_HLD_1, right=df_Effectif_HLD, how='left' , on=['Date_Format_1','CUID'])
df_Production_HLD_1.dropna(subset=['Matricule'], inplace=True)
df_Production_HLD_1 = df_Production_HLD_1[['Date_Format_1','CUID','Total_Logge','Duree_retrait_autres','Duree_retrait_formation','Duree_retrait_management','Duree_retrait_pause']]

df_Production_HLD_2 = df_Base_Data_agent_perf_Evita.loc[ (df_Base_Data_agent_perf_Evita['Flux'] == 'Orange Money')]
df_Production_HLD_2 = df_Production_HLD_2[['Date_Format_1','CUID','Appels_traites' , 'Appels_courts' , 'Appels_abandon_sur_sonnerie' , 'Appels_rotation_sur_sonnerie' , 'Appels_mis_en_attente' , 'Duree_log_total' , 'Duree_log_face_au_client' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_traitement' , 'Duree_conversation_appels_entrants' , 'Duree_garde_appels_entrants' , 'Duree_pas_libre' , 'Duree_wrapup' , 'Duree_pause_entre_appels' , 'Duree_THC']]

df_Effectif_HLD = df_Effectif_Global_HLD[['Date_Format_1','CUID','Log_PRA']]
df_Production_HLD_3 = df_Base_PRA.loc[ (df_Base_PRA['Flux'] == 'Orange Money') ]
df_Production_HLD_3 = df_Production_HLD_3.drop(columns=['Datation' ,'NOM','PRENOM','AFFECTATION'])
df_Production_HLD_3 = pd.merge(left=df_Production_HLD_3, right=df_Effectif_HLD, how='left' , left_on=['Date_Format_1','MATRICULE'], right_on=['Date_Format_1','Log_PRA'])
df_Production_HLD_3 = df_Production_HLD_3[['Date_Format_1','CUID','H_PLA','H_REA','H_ABS']]
df_Production_HLD_3.loc[df_Production_HLD_3['CUID'].isnull(), 'CUID'] = 'Non Defini'

pivoted=pd.pivot_table(df_Production_HLD_3, values='H_REA', index =['Date_Format_1'] , aggfunc='sum')
df_PRA_Dates=pd.DataFrame(pivoted.to_records())
df_PRA_Dates= df_PRA_Dates[df_PRA_Dates['H_REA'] == 0]
df_PRA_Dates=df_PRA_Dates["Date_Format_1"].unique()

df_Production_HLD_3_1 = df_Production_HLD_3.copy()
df_Production_HLD_3_1.rename(columns={'H_PLA': 'H_PLA_ABS','H_REA': 'H_REA_ABS','H_ABS': 'H_ABS_ABS'}, inplace=True)
df_Production_HLD_3_1 = df_Production_HLD_3_1[~df_Production_HLD_3_1['Date_Format_1'].isin(df_PRA_Dates)]

df_Production_HLD_4 = df_Base_REAL.loc[ ((df_Base_REAL['Flux'] == 'Orange Money')) & (df_Base_REAL['Activite'] == 'Improd')]
df_Production_HLD_4 = pd.merge(left=df_Production_HLD_4, right=df_Effectif_HLD, how='left' , left_on=['Date_Format_1','MATRICULE'], right_on=['Date_Format_1','Log_PRA'])
df_Production_HLD_4 = df_Production_HLD_4[['Date_Format_1','CUID','DUREE']]

df_Production_HLD_5 = df_Base_Appels_Courts.loc[ (df_Base_Appels_Courts['Flux'] == 'Orange Money')  ]
df_Production_HLD_5.rename(columns={'Appels_Courts': 'Appels_Courts_Global' }, inplace=True)
df_Production_HLD_5 = df_Production_HLD_5[['Date_Format_1','CUID','Appels_Courts_Global']]

df_Production_HLD_6 = df_Base_Reit_HLD[['Datation', 'Date_Format_1', 'CUID' , 'exploitables' , 'Reit1h' , 'Reit1J' , 'Reit7J']]
df_Production_HLD_6['Datation']=df_Production_HLD_6['Datation'].values.astype(str)
df_Production_HLD_6['Datation'] = df_Production_HLD_6['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
df_Production_HLD_6.loc[(df_Production_HLD_6['Datation'] <= datetime.now() - timedelta(days=8)), 'exploitables_7'] = df_Production_HLD_6['exploitables']
df_Production_HLD_6.loc[(df_Production_HLD_6['Datation'] <= datetime.now() - timedelta(days=8)), 'Reit7J_7'] = df_Production_HLD_6['Reit7J']
df_Production_HLD_6 = df_Production_HLD_6.drop(columns=['Datation'])

dfg_Production_HLD = pd.concat([df_Production_HLD_1, df_Production_HLD_2, df_Production_HLD_3, df_Production_HLD_3_1, df_Production_HLD_4, df_Production_HLD_5,df_Production_HLD_6], join='outer')
t_table_Production_HLD = pd.pivot_table(dfg_Production_HLD, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
dfg_Production_HLD=pd.DataFrame(t_table_Production_HLD.to_records())

df_Effectif_HLD = df_Effectif_Global_HLD.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])
dfg_Production_HLD = pd.merge(left=dfg_Production_HLD, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Production_HLD = pd.merge(left=dfg_Production_HLD, right=df_Effectif_HLD, how='left' , on=['Date_Format_1','CUID'])

dfg_Production_HLD['Datation']=dfg_Production_HLD['Datation'].values.astype(str)
dfg_Production_HLD['Datation'] = dfg_Production_HLD['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

#table = bigquery.Table(Id_table_Production_HLD)
#bq.delete_table(table, not_found_ok=True)
#table = bq.create_table(table)
#job = bq.load_table_from_dataframe(dfg_Production_HLD, table, job_config=job_config)